In [ ]:
pip install telethon pandas requests langdetect


In [ ]:
pip install python-dotenv

In [ ]:
pip install nest_asyncio

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except ModuleNotFoundError:
    pass


Mounted at /content/drive


In [ ]:
import os, json, csv, time, re
import asyncio, requests, pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from urllib.parse import urlparse
from langdetect import detect, DetectorFactory
DetectorFactory.seed=0load_dotenv()


In [ ]:
from telethon import TelegramClient, errors
from telethon.tl.functions.channels import JoinChannelRequest


In [ ]:
from telethon.sync import TelegramClient
from getpass import getpass

In [ ]:
# expects API_ID, API_HASH, and PHONE in the environment
API_ID = os.getenv("API_ID")
API_HASH = os.getenv("API_HASH")
PHONE = os.getenv("PHONE")
if not all([API_ID, API_HASH, PHONE]):
    raise RuntimeError("API_ID, API_HASH and PHONE environment variables must be set")


In [ ]:
PROGRESS_JSON = os.getenv('PROGRESS_JSON', '/content/drive/MyDrive/telegram/progress.json')
RAW_CSV = os.getenv('RAW_CSV', '/content/drive/MyDrive/telegram/messages.csv')
CLEAN_CSV = os.getenv('CLEAN_CSV', '/content/drive/MyDrive/telegram/cleaned.csv')
SESSION_NAME = os.getenv('SESSION_NAME', 'tg_session')

In [ ]:
BFS_HOPS         = 2     # how many BFS layers
BFS_PER_CH_LIMIT = 400   # how many msgs to scan for forwards each discovered channel
BFS_MIN_SUBS     = 500   # subscriber threshold
BFS_LANG_SAMPLE  = 15    # how many msgs we sample for English
BFS_EN_RATIO     = 0.5   # 50% must be English

In [ ]:
channels_df = pd.read_csv('telegram_misinformation_channels_full.csv')
def _parse_subs(s):
    s = str(s).replace('+', '').strip()
    mult = 1
    if s.lower().endswith('k'):
        mult = 1000
        s = s[:-1]
    elif s.lower().endswith('m'):
        mult = 1000000
        s = s[:-1]
    try:
        return int(float(s) * mult)
    except ValueError:
        return 0
channels_df['subs_num'] = channels_df['Subscribers'].apply(_parse_subs)
seed_usernames = channels_df['Username'].tolist()
FOLLOWERS_DICT = channels_df.set_index('Username')['subs_num'].to_dict()

In [ ]:
CHANNELS = seed_usernames

In [ ]:
client=TelegramClient(SESSION_NAME,API_ID,API_HASH)
def load_progress():
  if os.path.exists(PROGRESS_JSON):
    with open(PROGRESS_JSON,'r')as f:return json.load(f)
  return {}


In [ ]:
def save_progress(d):
  os.makedirs(os.path.dirname(PROGRESS_JSON),exist_ok=True)
  with open(PROGRESS_JSON,'w')as f:json.dump(d,f)
  print("Progress saved.")


In [ ]:
def append_header_if_needed():
  ex=os.path.exists(RAW_CSV)
  os.makedirs(os.path.dirname(RAW_CSV),exist_ok=True)
  if not ex:
    with open(RAW_CSV,'w',newline='',encoding='utf-8')as f:
      csv.writer(f).writerow(['channel','msg_id','date','text','fwd_from','fwd_id'])


In [ ]:
async def join_if_needed(ent):
  if getattr(ent,'megagroup',False):
    try:await client(JoinChannelRequest(ent))
    except:pass


In [ ]:
async def scrape_channel(name,last_id):
  try:ent=await client.get_entity(name)
  except Exception as e:print("Skip",name,e);return last_id
  await join_if_needed(ent);mx=last_id;cnt=0
  async for msg in client.iter_messages(ent,min_id=last_id,reverse=False):
    cnt+=1;mx=max(mx,msg.id);fw,fwid=None,None
    if msg.forward:
      if msg.forward.chat:fw=msg.forward.chat.title or msg.forward.chat.username;fwid=getattr(msg.forward.chat,'id',None)
      elif msg.forward.sender:fw=msg.forward.sender.first_name or msg.forward.sender.username or'Private';fwid=getattr(msg.forward.sender,'id',None)
    with open(RAW_CSV,'a',newline='',encoding='utf-8')as f:
      csv.writer(f).writerow([name,msg.id,msg.date,msg.text or'',fw,fwid])
  print(name,"+%d msgs"%cnt,"up to",mx);return mx


In [ ]:
def run_scrape():
  asyncio.run(scrape_once())
  print("Scraping done.")


In [ ]:
def deduplicate(messages):
  out=[];seen=set()
  for m in messages:
    k=(m.get('fwd_id'),m.get('text'))
    if m.get('fwd_id'):
      if k in seen:continue
      seen.add(k)
    out.append(m)
  return out


In [ ]:
def unshorten_url(u,t=5):
  try:return requests.head(u,allow_redirects=True,timeout=t).url
  except:return u


In [ ]:
def expand_and_extract(m):
  txt=m['text'];urls=re.findall(r'https?://\S+',txt)
  dom=set()
  for u in urls:
    fin=unshorten_url(u);d=urlparse(fin).netloc
    dom.add(d)
    txt=txt.replace(u,fin)
  m['text']=txt;m['domains']=list(dom);return m


In [ ]:
def is_english(txt):
  if not txt.strip():return False
  try:return(detect(txt)=='en')
  except:return False


In [ ]:
def anonymize(m,user_map):
  if m.get('fwd_id')and m['fwd_from']and not str(m['fwd_from']).startswith(('Channel','@')):
    if m['fwd_id'] not in user_map:user_map[m['fwd_id']]=f"User{len(user_map)+1}"
    m['fwd_from']=user_map[m['fwd_id']];m['fwd_id']=None
  m['text']=re.sub(r'@[\w\d_]+','@USER',m['text'])
  return m


In [ ]:
def preprocess_data():
  if not os.path.exists(RAW_CSV):print("No raw CSV.");return
  df=pd.read_csv(RAW_CSV,names=['chan','msg_id','date','text','fwd_from','fwd_id'],header=0)
  msgs=df.to_dict('records');msgs=deduplicate(msgs)
  out=[];umap={}
  for m in msgs:
    m=expand_and_extract(m)
    if is_english(m['text']):m=anonymize(m,umap);out.append(m)
  pd.DataFrame(out).to_csv(CLEAN_CSV,index=False)
  print(f"Saved {len(out)} cleaned msgs to {CLEAN_CSV}")


In [ ]:
# Load existing progress data and merge new seeds
progress = load_progress()
for username in seed_usernames:
    if username not in progress:
        progress[username] = 0  # mark as not yet scraped


In [ ]:
# Persist updated progress
save_progress(progress)

In [ ]:
from langdetect import detect
async def _en_ratio(ent, sample=BFS_LANG_SAMPLE):
    en = 0
    async for m in client.iter_messages(ent, limit=sample):
        t = (m.message or '').strip()
        if t:
            try:  en += detect(t)=='en'
            except: pass
    return 0 if not sample else en/sample


In [ ]:
from telethon.tl.types import PeerChannel
async def _forward_sources(ent, limit=BFS_PER_CH_LIMIT):
    src = set()
    async for m in client.iter_messages(ent, limit=limit):
        fwd = m.forward
        if not fwd: continue
        pc  = fwd.chat or fwd.original_fwd
        if isinstance(pc, PeerChannel): src.add(pc.channel_id)
    return src


In [ ]:
async def _is_valid(ent):
    """
    Check if a Telegram entity meets minimum subscriber and English-ratio criteria.
    It uses CSV's `Subscribers` column as the primary subscriber count.
    """
    username = (getattr(ent, 'username', '') or '').strip()

    # Fetch subscriber count directly from CSV, defaulting to 0 if not found
    count = FOLLOWERS_DICT.get(username, 0)

    if count < BFS_MIN_SUBS:
        return False

    # Check English language ratio
    try:
        return (await _en_ratio(ent)) >= BFS_EN_RATIO
    except Exception:
        return False


In [ ]:
async def snowball_discovery():
    global CHANNELS
    prog, seeds = load_progress(), [c.lstrip('@') for c in CHANNELS]
    frontier, visited, validated = set(seeds), set(seeds), []

    for hop in range(1, BFS_HOPS+1):
        nxt = set()
        print(f'· hop {hop}  frontier={len(frontier)}')
        for h in list(frontier):
            try: ent = await client.get_entity(h)
            except: continue
            if not await _is_valid(ent): continue
            validated.append(h)
            for cid in await _forward_sources(ent):
                try:
                    e = await client.get_entity(PeerChannel(cid))
                    if e.username and e.username not in visited:
                        visited.add(e.username); nxt.add(e.username)
                except: pass
        frontier = nxt
    for v in validated:
        if v not in prog: prog[v] = 0
    save_progress(prog)
    globals()['CHANNELS'] = list(set(CHANNELS)|set(prog))
    print('validated:',len(validated),' | total CHANNELS:',len(CHANNELS))


In [ ]:
async def scrape_once():
    await client.start(phone=PHONE)
    await snowball_discovery()          # ①  discover / merge
    last = load_progress(); append_header_if_needed()
    for c in CHANNELS:
        last[c] = await scrape_channel(c, last.get(c,0))
    save_progress(last); await client.disconnect()


In [ ]:
def main():
  run_scrape()
  preprocess_data()
  print("Done.")


In [ ]:
# main()